In [1]:
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn import preprocessing
import joblib
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from osgeo import gdal, gdalconst, ogr
from IPython import display

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_name, feature):
        self.file_name = file_name
        self.feature = feature
#         self.transforms = transforms
        data = pd.read_csv(self.file_name)
        data = data.dropna(axis = 0)
        self.x = data[self.feature].values
        self.y = data['label'].values
        self.len = len(data)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        x_input = self.x[idx]
        label = self.y[idx] - 1
        return x_input, label

In [3]:
class Net(nn.Module):
    def __init__(self, nfeature):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(nfeature, 500)
        self.lin2 = nn.Linear(500, 1000)
        self.lin3 = nn.Linear(1000, 300)
        self.lin4 = nn.Linear(300, 2)
        self.bn1 = nn.BatchNorm1d(nfeature)
        self.bn2 = nn.BatchNorm1d(300)
        self.dropout = nn.Dropout(p=0.5)
#         self.bn2 = nn.BatchNorm1d(500)
#         self.bn3 = nn.BatchNorm1d(300)
        
    def forward(self, x):
        x = self.bn1(x)
        x = F.relu(self.lin1(x))
        x = self.dropout(x)
        x = F.relu(self.lin2(x))
        x = self.dropout(x)
        x = F.relu(self.lin3(x))
        x = self.bn2(x)
        x = self.lin4(x)
        x = torch.sigmoid(x)
        return x

In [4]:
def get_loader(feature, val):
    val_data = []
    train_data = MyDataset('train_ne_all.csv', feature)
    test_data = MyDataset('test_ne_all.csv', feature)
    if val:
        val_data = MyDataset('label1_regress.csv', feature)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
    return train_loader, test_loader, val_loader

In [5]:
def train(train_loader, model, optimizer, loss_func):
    model.train()
    losses = []
    TOTAL_EPOCHS = 50
    for epoch in range(TOTAL_EPOCHS):
        train_loss = 0.
        train_acc = 0.
        for i, (x, y) in enumerate(train_loader):
            x = x.float().to(device)
            y = y.long().to(device)
#             print(x.shape)
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_func(outputs, y)
            loss.backward()
            optimizer.step()
            losses.append(loss.cpu().data.item())
        if (epoch + 1) % 10 == 0:
            print(("epochs:{0}/{1}, loss:{2}".format(epoch, TOTAL_EPOCHS, np.mean(losses))))

In [6]:
def test(test_loader, model, optimizer, loss_func):
    model.eval()
    correct = 0
    total = 0
    conf = np.zeros((2, 2))
    for i,(x, y) in enumerate(test_loader):
        x = x.float().to(device)
        y = y.long()
        outputs = model(x).cpu()
    #     print(outputs)
        _, predicted = torch.max(outputs.data, 1)
    #     print(predicted)
    #     print(y)
        total += y.size(0)
        correct += (predicted == y).sum()
        conf = conf + confusion_matrix(predicted, y)
    return conf

In [7]:
def get_result(conf):
    print(conf)
    recall = conf[0][0] / conf[:, 0:1].sum() * 100
    precision = conf[0][0] / conf[0].sum() * 100
    accuracy = (conf[0][0] + conf[1][1]) / conf.sum() * 100
    F1 = 2 * recall * precision / (recall + precision)
    print('recall : %.4f %%' % recall)
    print('precision : %.4f %%' % precision)
    print('accuracy : %.4f %%' % accuracy)
    print('F1 : %.4f' % F1)
    print([recall, precision, accuracy, F1])
    return [recall, precision, accuracy, F1]

In [8]:
device = torch.device("cpu")
if torch.cuda.is_available():
        device = torch.device("cuda")

In [9]:
class nnn:
    def __init__(self, feature_list):
        print('nn')
        self.feature_list = feature_list
        self.val = val
    
    
    def nn_result(self):
        result_ABC = []
        for feature in self.feature_list:
            nfeature = len(feature)
            train_loader, test_loader, val_loader = get_loader(feature, self.val)
            model = Net(nfeature).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
            loss_func = torch.nn.CrossEntropyLoss()
            train(train_loader, model, optimizer, loss_func)
            conf = test(test_loader, model, optimizer, loss_func)
            if self.val:
                conf_val = test(val_loader, model, optimizer, loss_func)
            result = get_result(conf)
            result_ABC.append(result)
#             result_val = get_result(conf_val)
        return result_ABC

In [10]:
list_00 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',                
           'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',                
           'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','PR_NDVI_1','PR_NDVI_2','TXU_ENT','TXU_asm','area','LT_mag',
           'LT_dur','TXU_HOM','TXU_CON','ndvi_rvalue','ndmi_rvalue','nbr_rvalue','ndvi_slope','ndmi_slope','nbr_slope']

list_01 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',                
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',                
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','PR_NDVI_1','PR_NDVI_2','TXU_ENT','TXU_asm','area','LT_mag',                
            'LT_dur','TXU_HOM','TXU_CON']

list_02 = ['PT_NDMI_0','PR_NDMI_1','PR_NDMI_2','PT_NBR_0','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PR_NDVI_1','PR_NDVI_2','TXU_ENT',                
            'TXU_asm','area','LT_mag', 'LT_dsnr','LT_dur','TXU_HOM','TXU_CON','ndvi_rvalue','ndmi_rvalue','nbr_rvalue',                
            'ndvi_slope','ndmi_slope','nbr_slope']

list_03 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',                
           'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',                
           'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','PR_NDVI_1','PR_NDVI_2','TXU_ENT','TXU_asm','area','LT_mag',
           'LT_dur','TXU_HOM','TXU_CON','NDVI_rvalue','NDMI_rvalue','NBR_rvalue','NDVI_slope','NDMI_slope','NBR_slope']

list_04 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PR_NDMI_1','PR_NDMI_2','PT_NBR_0',                
            'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PR_NBR_1','PR_NBR_2','PT_NDVI_0','PT_NDVI_1',                
            'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5','PR_NDVI_1','PR_NDVI_2','TXU_ENT','TXU_asm','area','LT_mag']

list_05 = ['PT_NDMI_0','PT_NDMI_1','PT_NDMI_2','PT_NDMI_3','PT_NDMI_4','PT_NDMI_5','PT_NBR_0',                
           'PT_NBR_1','PT_NBR_2','PT_NBR_3','PT_NBR_4','PT_NBR_5','PT_NDVI_0','PT_NDVI_1',                
           'PT_NDVI_2','PT_NDVI_3','PT_NDVI_4','PT_NDVI_5'      
            ,'area','LT_mag','NDVI_rvalue','NDMI_rvalue','NBR_rvalue','NDVI_slope','NDMI_slope','NBR_slope']

feature_list1 = [list_00, list_01, list_02]
feature_list2 = [list_03, list_04, list_05]

In [11]:
val = 0
nnn = nnn([list_05]).nn_result()

nn
epochs:9/50, loss:0.5334405083190366
epochs:19/50, loss:0.5176890632222751
epochs:29/50, loss:0.5073580297772251
epochs:39/50, loss:0.5001015211006409
epochs:49/50, loss:0.4941776392181513
[[2099.  350.]
 [ 639. 2312.]]
recall : 76.6618 %
precision : 85.7085 %
accuracy : 81.6852 %
F1 : 80.9331
[76.6617969320672, 85.70845242956308, 81.68518518518518, 80.93310198573357]


In [12]:
nnn

[[76.6617969320672, 85.70845242956308, 81.68518518518518, 80.93310198573357]]

In [2]:
from torch.utils.benchmark import Timer

results = []
for num_threads in [1, 2, 4]:
    timer = Timer(
        stmt="torch.add(x, y, out=out)",
        setup="""
            n = 1024
            device = torch.device("cpu")
            if torch.cuda.is_available():
                device = torch.device("cuda")
            x = torch.ones((n, n)).to(device)
            y = torch.ones((n, 1)).to(device)
            out = torch.empty((n, n)).to(device)
        """,
        num_threads=num_threads,
    )
    results.append(timer.blocked_autorange(min_run_time=5))
    print(
        f"{num_threads} thread{'s' if num_threads > 1 else ' ':<4}"
        f"{results[-1].median * 1e6:>4.0f} us   " +
        (f"({results[0].median / results[-1].median:.1f}x)" if num_threads > 1 else '')
    )

C:\Users\kikyo\anaconda3\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


1 thread      14 us   
2 threads     14 us   (1.0x)
4 threads     13 us   (1.0x)
8 threads     13 us   (1.0x)


In [1]:
import torch
torch.cuda.is_available()

True